Aggregate image level probability to restaurants and show score

In [1355]:
directory = '/result/cnn.csv'
bus_dir  = '/pic_label_subbybus_test.csv'
feature2 = '/result/colorf2log.csv'

In [1356]:
import pandas as pd
import numpy as np

In [1357]:
bus_info = pd.read_csv(bus_dir, header = 0)
prob = pd.read_csv(directory, header = 0)
prob.columns = ['photo_id', 'label0', 'label1', 'label2', 'label3', 
                'label4', 'label5', 'label6', 'label7', 'label8']
prob2 = pd.read_csv(feature2, header = 0)
prob2.columns = ['photo_id', 'label2_0', 'label2_1', 'label2_2', 
                 'label2_3', 'label2_4', 'label2_5', 'label2_6', 'label2_7', 'label2_8']


In [1193]:
######feature combine#####

In [1194]:
proball = pd.merge(prob, prob2, on = 'photo_id', how = 'left')

In [1040]:
proball['label0'] = 0.5*proball['label0'] + 0.5*proball['label2_0']
proball['label1'] = 0.5*proball['label1'] + 0.5*proball['label2_1']
proball['label2'] = 0.5*proball['label2'] + 0.5*proball['label2_2']
proball['label3'] = 0.5*proball['label3'] + 0.5*proball['label2_3']
proball['label4'] = 0.5*proball['label4'] + 0.5*proball['label2_4']
proball['label5'] = 0.5*proball['label5'] + 0.5*proball['label2_5']
proball['label6'] = 0.5*proball['label6'] + 0.5*proball['label2_6']
proball['label7'] = 0.5*proball['label7'] + 0.5*proball['label2_7']
proball['label8'] = 0.5*proball['label8'] + 0.5*proball['label2_8']

In [1041]:
prob = proball.loc[:, 'photo_id':'label8']

In [1013]:
#######################

In [1358]:
prob_only = prob.copy()
prob_only = prob_only.loc[:, 'label0':'label8']
prob_only[prob_only<0.5] = 0
prob_only[prob_only>0.5] = 1

In [1359]:
sum_all = pd.DataFrame(prob['photo_id']).join(prob_only)

In [1360]:
sum_all[0:5]

,photo_id,label0,label1,label2,label3,label4,label5,label6,label7,label8
0,80,0,0,0,0,0,0,0,0,0
1,95,0,1,1,0,0,1,1,0,1
2,130,0,1,1,0,0,1,1,0,1
3,143,0,1,1,0,0,1,1,0,1
4,296,0,1,1,0,0,1,1,0,1


In [1361]:
mergeall = pd.merge(prob,bus_info,on='photo_id', how='left')
mergeall[['photo_id']] =1
merge_sum = pd.merge(sum_all,bus_info,on='photo_id', how='left')

In [1362]:
mergeall[0:10]

,photo_id,label0,label1,label2,label3,label4,label5,label6,label7,label8,business_id,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8
0,1,0,0,0,0,0,0,0,0,0,1114,0,1,1,0,1,1,1,1,0
1,1,0,1,1,0,0,1,1,0,1,3065,0,1,1,0,1,1,1,1,0
2,1,0,1,1,0,0,1,1,0,1,2687,0,1,1,0,0,1,1,1,0
3,1,0,1,1,0,0,1,1,0,1,3979,0,1,1,1,0,1,1,1,0
4,1,0,1,1,0,0,1,1,0,1,1114,0,1,1,0,1,1,1,1,0
5,1,0,1,1,0,0,1,1,0,1,1160,1,1,1,0,0,1,1,0,1
6,1,0,1,1,0,0,1,1,0,1,1931,0,0,1,0,0,0,1,0,0
7,1,0,1,1,0,0,1,1,0,1,2688,0,1,1,1,0,1,1,1,0
8,1,0,1,1,0,0,1,1,0,1,1114,0,1,1,0,1,1,1,1,0
9,1,0,1,1,0,0,1,1,0,1,2315,0,1,1,1,1,1,1,1,0


In [1324]:
####test theshold by individual 

In [1325]:
#mergeall['label0']

In [1363]:
get_average= mergeall.groupby(['business_id']).mean().reset_index()
get_sum= merge_sum.groupby(['business_id']).sum().reset_index()

In [1327]:
# print get_sum[0:5]
# print get_average[0:5]

In [1364]:
# get_average['label0'][get_average['label0']<0.3] = 0 
# get_average['label0'][get_average['label0']>0.3] = 1 
# get_average['label3'][get_average['label3']<0.4] = 0 
# get_average['label3'][get_average['label3']>0.4] = 1
# get_average['label7'][get_average['label3']<0.6] = 0 
# get_average['label7'][get_average['label3']>0.6] = 1
get_average[get_average<0.5] = 0
get_average[get_average>=0.5] = 1

In [1365]:
get_sum[get_sum>=1] = 1
get_sum[get_sum<1] = 0

In [1351]:
#get_average[0:5]

In [1366]:
from sklearn import metrics

In [1367]:
for i in range(9):
    pred_y = get_average['label%d'%i]
    actual_y = get_average['label_%d'%i]
    score =metrics.accuracy_score(pred_y,actual_y)
    f0 = metrics.f1_score(pred_y,actual_y)
    print score,f0

0.636363636364 0.0
0.565656565657 0.722580645161
0.545454545455 0.705882352941
0.515151515152 0.0
0.686868686869 0.0
0.676767676768 0.807228915663
0.747474747475 0.85549132948
0.616161616162 0.0
0.575757575758 0.730769230769


In [1368]:
for i in range(9):
    pred_y = get_sum['label%d'%i]
    actual_y = get_sum['label_%d'%i]
    score =metrics.accuracy_score(pred_y,actual_y)
    f0 = metrics.f1_score(pred_y,actual_y)
    print score,f0

0.636363636364 0.0
0.565656565657 0.722580645161
0.545454545455 0.705882352941
0.525252525253 0.0408163265306
0.686868686869 0.0
0.676767676768 0.807228915663
0.747474747475 0.85549132948
0.616161616162 0.0
0.575757575758 0.730769230769


In [1369]:
pred_y_avg = get_average.loc[:, 'label0':'label8']
true_y_avg = get_average.loc[:, 'label_0':'label_8']
pred_y_sum = get_sum.loc[:, 'label0':'label8']
true_y_sum = get_sum.loc[:, 'label_0':'label_8']

pred_y_avg['label3'] = pred_y_sum['label3']
pred_y_avg['label0'] = pred_y_sum['label0']

In [1370]:
def mlb_f1(y_true, y_pred):
    y_true = set(y_true)
    y_pred = set(y_pred)
    tp = len(y_true & y_pred)
    #fp = len(y_pred-y_true)
    #fn = len(y_true-y_pred)
    fp = len(y_true-y_pred)
    fn = len(y_pred-y_true)
    if tp == 0 and fp == 0:
        return 0.0
    else:
        p = float(tp)/(tp+fp)
        r = float(tp)/(tp+fn)
        return 2*p*r/(p+r) if tp!=0 else 0.
def mean_f1(y_true, y_pred):
    score_list = map(lambda x: mlb_f1(x[0],x[1]), zip(y_true, y_pred))
    return sum(score_list)/len(score_list)

In [1371]:
y_true = [
              [1, 2],
              [3, 4, 5],
              [6],
              [7]]
y_pred = [
              [1, 2, 3, 9],
              [3, 4],
              [6, 12],
              [1]
    ]
print mean_f1(y_true, y_pred)

0.533333333333


In [1372]:
y_pred_avg = [[i for i in range(len(arr)) if arr[i]!=0] for arr in np.array(pred_y_avg)]
y_true_avg = [[i for i in range(len(arr)) if arr[i]!=0] for arr in np.array(true_y_avg)]
y_pred_sum = [[i for i in range(len(arr)) if arr[i]!=0] for arr in np.array(pred_y_sum)]
y_true_sum = [[i for i in range(len(arr)) if arr[i]!=0] for arr in np.array(true_y_sum)]

In [1373]:
mean_f1(y_true_avg, y_pred_avg)

0.6167369106763042

In [1374]:
mean_f1(y_true_sum, y_pred_sum)

0.6167369106763042

feature combine